# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 

import re
import pickle
import time

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet');

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michelangelo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michelangelo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michelangelo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database - change path accordingly
engine = create_engine('sqlite:///../data/data_db/DisasterReponses.db')
df = pd.read_sql("SELECT * FROM DisasterReponses", engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.info()

In [4]:
# The messages will be the features, while the categories will be the target variables
X = df.message.to_numpy() 
y = df[df.columns[4:]].to_numpy()

In [5]:
# Check size of dataset
print("Complete feature dataset size = ", X.shape, "; complete target variables size = ", y.shape)

Complete feature dataset size =  (26216,) ; complete target variables size =  (26216, 36)


In [6]:
# Check what's in X, y
X[0], y[0]

('Weather update - a cold front from Cuba that could pass over Haiti',
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [8]:
# Finally, save the names of the category labels
category_names = list(df.columns[4:])
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens_raw = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens_raw if (word not in stopwords.words('english'))]

    return tokens

In [10]:
# Test function for the first 5 messages
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Split data in test/train


In [11]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# Check size of datasets - default split is 75% train / 25% test
print("Training feature dataset size = ", X_train.shape, "; training target variables size = ", y_train.shape)
print("Testing feature dataset size = ", X_test.shape, "; testing target variables size = ", y_test.shape)

Training feature dataset size =  (19662,) ; training target variables size =  (19662, 36)
Testing feature dataset size =  (6554,) ; testing target variables size =  (6554, 36)


### 4. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 5.a. Train pipeline

In [ ]:
# train classifier, keeping track of time
t0= time.clock()

pipeline.fit(X_train, y_train);

t1 = time.clock() - t0

In [ ]:
print('elapsed time = ', t1)

### 5.b. Load saved pipeline

In case the pipeline was saved previously you can load it here

In [14]:
# Load dictionary 
filename = '../models/models_files/random_forest_pipeline_1.pickle'
pipeline_dict = pickle.load(open(filename, 'rb'))

In [15]:
# Get content
X_train = pipeline_dict['X_train']
y_train = pipeline_dict['y_train']
X_test = pipeline_dict['X_test']
y_test = pipeline_dict['y_test']
pipeline = pipeline_dict['pipeline']

### 6. Test your pipeline
 - Predict outcomes based on test data
 - Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [17]:
# display example of results 
print(classification_report(y_test[:,0], y_pred[:,0]))

              precision    recall  f1-score   support

           0       0.70      0.43      0.53      1529
           1       0.84      0.94      0.89      4977
           2       0.35      0.35      0.35        48

    accuracy                           0.82      6554
   macro avg       0.63      0.58      0.59      6554
weighted avg       0.80      0.82      0.80      6554



In [28]:
# Iterate and print on screen and file
for ind_1 in range(y_pred.shape[1]):
    print('-----------------------------------------------------------------------------------------')
    print('Label = ', category_names[ind_1])
    c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1], output_dict=True, zero_division=0)
    kk = list(c_rep.keys())
    for ind_2 in range(len(c_rep) - 3):
        print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
              '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']), 
              '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
              '; support =', c_rep[kk[ind_2]]['support'])

-----------------------------------------------------------------------------------------
Label =  related
Value =  0 : precision =  0.70 ; recall =  0.43 ; f1-s = 0.53 ; support = 1529
Value =  1 : precision =  0.84 ; recall =  0.94 ; f1-s = 0.89 ; support = 4977
Value =  2 : precision =  0.35 ; recall =  0.35 ; f1-s = 0.35 ; support = 48
-----------------------------------------------------------------------------------------
Label =  request
Value =  0 : precision =  0.90 ; recall =  0.98 ; f1-s = 0.94 ; support = 5433
Value =  1 : precision =  0.82 ; recall =  0.48 ; f1-s = 0.60 ; support = 1121
-----------------------------------------------------------------------------------------
Label =  offer
Value =  0 : precision =  1.00 ; recall =  1.00 ; f1-s = 1.00 ; support = 6526
Value =  1 : precision =  0.00 ; recall =  0.00 ; f1-s = 0.00 ; support = 28
-----------------------------------------------------------------------------------------
Label =  aid_related
Value =  0 : precisio

You can run the following cells if you want to save the previous report on a text file

In [18]:
# Uncomment this (or add the library in the import section) if you want to save results in a text file including 
# the date of the test
# from datetime import datetime

In [29]:
# You can change this accordingly to your wish
output_file_path = '../models/models_files/'
# You can pick the format for the score file name
# output_file_name = 'score_results_' + datetime.now().strftime('%Y_%m_%d') + '.txt' 
output_file_name = 'score_results_random_forest_pipeline_1.txt'
output_file_full_name = output_file_path + output_file_name

In [30]:
# Iterate and print on file
with open(output_file_full_name, "w") as text_file:
    for ind_1 in range(y_pred.shape[1]):
        print('-----------------------------------------------------------------------------------------', file=text_file)
        print('Label = ', category_names[ind_1], file=text_file)
        c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1], output_dict=True, zero_division=0)
        kk = list(c_rep.keys())
        for ind_2 in range(len(c_rep) - 3):
            print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
                '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']),
                '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
                '; support =', c_rep[kk[ind_2]]['support'], file=text_file)

### 7. Export your model as a pickle file

If you have just now created the pipeline, save it and the data in a pickle file

In [ ]:
# Create dictionary for pickle file
pipeline_dict_2 = {'X_train':X_train,
                    'y_train':y_train,
                    'X_test':X_test,
                    'y_test':y_test,
                    'pipeline':pipeline}

In [ ]:
# Save dictionary
filename = '../models/models_files/random_forest_pipeline_1.pickle'
pickle.dump(classifier_dict_2, open(filename, 'wb'))

### 8. Improve your model
Use grid search to find better parameters. 

In [31]:
# Find what are the parameters of the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7ff9aaec0510>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7ff9aaec0510>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [ ]:
# Define grid search parameters
parameters = {
#         'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
#         'vect__max_features': (None, 5000, 10000),
#         'tfidf__use_idf': (True, False),
#         'clf__estimator__n_estimators': [50, 100, 200],
#         'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

**NOTE:** Running the grid search on the full parameter space can take a **_lot_** of time

In [ ]:
# train classifier, keeping track of time
t0= time.clock()

cv.fit(X_train, y_train);

t1 = time.clock() - t0

### 9. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

###  10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.